# 라이브러리 불러오기

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# VectorStore
- VectorStore는 자연어 -> 숫자 처리 한 후 이들을 저장하는 벡터 저장소
- 벡터 저장소는 임베딩된 데이터를 인덱싱하여, input으로 받아들이는 query와 유사도를 빠르게 출력
- 대표적으로 FAISS, Chroma 가 존재

# Chroma
- Chroma는 대표적인 오픈소스 벡터 저장소
- 기본적으로 VectorStore는 벡터를 일시적으로 저장
- 텍스트와 임베딩 함수를 지정 하여 from_documents() 함수에 전달
- 지정된 임베딩 함수를 통해 텍스트를 벡터로 변환 하고 임시 db 생성
- similarity_search() 함수를 쿼리로 지정해 두면, 이를 바탕 가장 벡터 유사도가 높은 벡터를 찾고, 이를 자연어 형태로 출력

In [10]:
# !pip install chromadb transformers sentence_transformers tiktoken
# !pip install pypdf

In [2]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [3]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader, PyPDFLoader

In [4]:
loader = PyPDFLoader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()
len(pages)

18

In [5]:
text_splitteer = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=tiktoken_len
)
docs = text_splitteer.split_documents(pages)

In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "jhgan/ko-sbert-nli"

model_name = "JaeyeonKang/CCK-v1.0.0-DPO"
model_kwargs = {"device":"cuda:0"}
encode_kwargs = {"normalize_embeddings":True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

No sentence-transformers model found with name /home/restful3/.cache/torch/sentence_transformers/JaeyeonKang_CCK-v1.0.0-DPO. Creating a new one with MEAN pooling.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_7591/1635014637.py:8 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_7591/1635014637.py'                         │
│                                                                                                  │
│ /home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/langchain_community/embedding │
│ s/huggingface.py:232 in __init__                                                                 │
│                                                                                                  │
│   229 │   │   │   │   "Please install it with `pip install sentence_transformers`."              │
│   230 │   │   │   ) from exc                                                                     │
│   231 │   │                                                                                      │
│ ❱ 232 │   │   self.client = sentence_transformers.SentenceTransformer(                           │
│   233 │   │   │   self.model_name, cache_folder=self.cache_folder, **self.model_kwargs           │
│   234 │   │   )                                                                                  │
│   235 │   │   if "-zh" in self.model_name:                                                       │
│                                                                                                  │
│ /home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/sentence_transformers/Sentenc │
│ eTransformer.py:97 in __init__                                                                   │
│                                                                                                  │
│    94 │   │   │   if os.path.exists(os.path.join(model_path, 'modules.json')):    #Load as Sen   │
│    95 │   │   │   │   modules = self._load_sbert_model(model_path)                               │
│    96 │   │   │   else:   #Load with AutoModel                                                   │
│ ❱  97 │   │   │   │   modules = self._load_auto_model(model_path)                                │
│    98 │   │                                                                                      │
│    99 │   │   if modules is not None and not isinstance(modules, OrderedDict):                   │
│   100 │   │   │   modules = OrderedDict([(str(idx), module) for idx, module in enumerate(modul   │
│                                                                                                  │
│ /home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/sentence_transformers/Sentenc │
│ eTransformer.py:806 in _load_auto_model                                                          │
│                                                                                                  │
│   803 │   │   Creates a simple Transformer + Mean Pooling model and returns the modules          │
│   804 │   │   """                                                                                │
│   805 │   │   logger.warning("No sentence-transformers model found with name {}. Creating a ne   │
│ ❱ 806 │   │   transformer_model = Transformer(model_name_or_path)                                │
│   807 │   │   pooling_model = Pooling(transformer_model.get_word_embedding_dimension(), 'mean'   │
│   808 │   │   return [transformer_model, pooling_model]                                          │
│   809                                                                                            │
│                                                                                                  │
│ /home/restful3/anaconda3/envs/trading/lib/python3.10/site-packages/sentence_transformers/models/ │
│ Transformer.py:28 in __init__                              

In [7]:
db = Chroma.from_documents(docs, hf)

In [8]:
query = "6대 먹거리 산업은 무엇인가?"
docs = db.similarity_search(query)

In [9]:
print(docs[0].page_content)

혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 7 |▶5G 이동통신 시스템 산업의 value chain 은 ‘칩셋 및 장비 → 5G 이동통신 단말 및 기지국 → 
5G 이동통신 네트워크 → 이동통신 서비스 ’로 구성되며 , 동 산업은 ①전방산업에 대한 파급효과가  
큰 산업, ②진입장벽이 높은 산업, ③지속적인 R&D가 요구되는 산업 등의 특징을 가짐
￮방송통신 서비스 , 사물인터넷 , 모바일 뱅킹, 전자상거래 , 건설, 에너지 , 의료, 국방, 조선, 물류, 
자동차 등의 다양한 산업을 대상으로 하는 등 전방산업에 파급효과가 큰 특징이 있음
￮초기시장 선점을 통한 높은 진입장벽이 형성되어 독과점 현상이 뚜렷한 산업분야로 , 기존 4G LTE 
와의 호환성 때문에 5G 이동통신 시스템 시장에서도 기존 사업자와 계약을 진행하는 경향이 있음
￮전체 네트워크 설계역량이 경쟁력의 핵심요소이며 , 지속적인 제품 개발능력과 고객을 만족시키기 
위한 마케팅 활동 등에서 차별적 경쟁우위를 확보하는 것이 중요한 경쟁요소임
▶시장조사전문기관 Technavio 에 따르면 세계 5G 이동통신 시스템 시장규모는 2020년 494.4 억 
달러에서 연평균 32.0% 성장하여 2025년에는 1,982 억 달러의 시장을 시현할 것으로 전망됨
￮5G 상용화 서비스가 본격화됨에 따라 국내 5G 통신서비스 가입자 수는 매년 증가하고 있는 추세임
      * (‘19년 말)4,668,154 명 → (’20년 말)11,851,373 명  → (’21년 말)20,915,176 명 (출처: 과학기술정보통신부 )
[세계 5G 이동통신 시스템 시장규모 ]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분 2020 2021 2022 2023 2024 2025CAGR
(2020-2025)
5G이동통신 
시스템494.4 645.1 815.5 1,063.5 1,429

- 대부분의 경우에 활용하고자 하는 문서를 나만의 디스크에 저장하고 필요 할 때 마다 호출
- persist() 함수로 벡터 저장소를 로컬에 저장, Chroma 객체를 선언 시 로컬 저장소 경로를 지정하여 다시 불러 올 수 있음

- save to disk

In [51]:
db2 = Chroma.from_documents(docs, hf, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

- load from disk

In [52]:
db3 = Chroma(persist_directory="./chroma_db", embedding_function=hf)
docs = db3.similarity_search(query)
print(docs[0].page_content)

혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 7 |▶5G 이동통신 시스템 산업의 value chain 은 ‘칩셋 및 장비 → 5G 이동통신 단말 및 기지국 → 
5G 이동통신 네트워크 → 이동통신 서비스 ’로 구성되며 , 동 산업은 ①전방산업에 대한 파급효과가  
큰 산업, ②진입장벽이 높은 산업, ③지속적인 R&D가 요구되는 산업 등의 특징을 가짐
￮방송통신 서비스 , 사물인터넷 , 모바일 뱅킹, 전자상거래 , 건설, 에너지 , 의료, 국방, 조선, 물류, 
자동차 등의 다양한 산업을 대상으로 하는 등 전방산업에 파급효과가 큰 특징이 있음
￮초기시장 선점을 통한 높은 진입장벽이 형성되어 독과점 현상이 뚜렷한 산업분야로 , 기존 4G LTE 
와의 호환성 때문에 5G 이동통신 시스템 시장에서도 기존 사업자와 계약을 진행하는 경향이 있음


- 쿼리와 유사한 문서(청크)를 불러올 때, 유사도를 함께 제공하는 함수 similarity_search_with_score() 제공
- 내가 얻은 유사한 문장들의 유사도를 비교 가능
- 특정 유사도 이상의 문서만 출력 가능

In [55]:
docs = db3.similarity_search_with_relevance_scores(query, k=3)

print(f"가장 유사한 문서 : {docs[0][0].page_content}")
print(f"문서 유사도 : {docs[0][1]}")

가장 유사한 문서 : 혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 7 |▶5G 이동통신 시스템 산업의 value chain 은 ‘칩셋 및 장비 → 5G 이동통신 단말 및 기지국 → 
5G 이동통신 네트워크 → 이동통신 서비스 ’로 구성되며 , 동 산업은 ①전방산업에 대한 파급효과가  
큰 산업, ②진입장벽이 높은 산업, ③지속적인 R&D가 요구되는 산업 등의 특징을 가짐
￮방송통신 서비스 , 사물인터넷 , 모바일 뱅킹, 전자상거래 , 건설, 에너지 , 의료, 국방, 조선, 물류, 
자동차 등의 다양한 산업을 대상으로 하는 등 전방산업에 파급효과가 큰 특징이 있음
￮초기시장 선점을 통한 높은 진입장벽이 형성되어 독과점 현상이 뚜렷한 산업분야로 , 기존 4G LTE 
와의 호환성 때문에 5G 이동통신 시스템 시장에서도 기존 사업자와 계약을 진행하는 경향이 있음
문서 유사도 : 0.2869551504637776


#  FAISS
- Facebook AI의 유사성 검색(FAISS)는 고밀도 벡터의 효율적인 유사성 검색 및 클러스터링을 위한 라이브러리
- 모든 키기의 벡터 집합에서 검색하는 알고리즘이 포함되어 있으며
- RAM에 맞지 않을 수 있는 벡터 까지 검색 가능
- 평가 및 매게변수 조정을 위한 지원 코드 포함

In [56]:
# !pip install faiss-gpu

In [57]:
from langchain.vectorstores import FAISS

In [67]:
loader = PyPDFLoader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitteer = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=tiktoken_len
)
docs = text_splitteer.split_documents(pages)

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {"device":"cuda:0"}
# model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}
ko = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [68]:
db = FAISS.from_documents(docs, ko)

In [69]:
query = "인공지능 산업구조는 어떻게 구성되어 있어?"
docs = db.similarity_search(query)
print(docs[0].page_content)

혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 9 |▶다양한 데이터나 복잡한 자료 속에서 핵심적인 특징을 요약하는 ①데이터 추상화 기술, 방대한 지식
체계를 이용하는 ②빅데이터 기술, 빅데이터를 처리하기 위한 ③고성능 컴퓨팅 기술이 인공지능 
구현의 핵심임
￮데이터를 추상화하는 방법은 크게 인공신경망 (ANN), 심층신경망 (DNN), 합성곱신경망 (CNN) 및 
순환신경망 (RNN) 등으로 구분됨
[인공지능 데이터 추상화 기술]
구분 특징 장점 단점
인공신경망 (ANN)
Artificial Neural 
Network사람의 신경망 원리와 구조를 모방하여 만든 
기계학습 알고리즘으로 , 입력층 , 출력층 , 은닉층
으로 구성모든 비선형 함수 
학습이 가능알고리즘을 최적화
하기 어려운 학습 
환경 발생
심층신경망 (DNN)
Deep Neural 
Network입력층과 출력층 사이에 2개 이상의 은닉층
들로 이뤄진 인공신경망ANN의 문제점 개선학습환경에 따라 
높은 시간 복잡도  
문제 발생
합성곱신경망 (CNN)


In [71]:
docs_and_scores = db.similarity_search_with_score(query) #점수가 낮을 수록 거리가 가까움
docs_and_scores

[(Document(page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2 호 | 9 |▶다양한 데이터나 복잡한 자료 속에서 핵심적인 특징을 요약하는 ①데이터 추상화 기술, 방대한 지식\n체계를 이용하는 ②빅데이터 기술, 빅데이터를 처리하기 위한 ③고성능 컴퓨팅 기술이 인공지능 \n구현의 핵심임\n￮데이터를 추상화하는 방법은 크게 인공신경망 (ANN), 심층신경망 (DNN), 합성곱신경망 (CNN) 및 \n순환신경망 (RNN) 등으로 구분됨\n[인공지능 데이터 추상화 기술]\n구분 특징 장점 단점\n인공신경망 (ANN)\nArtificial Neural \nNetwork사람의 신경망 원리와 구조를 모방하여 만든 \n기계학습 알고리즘으로 , 입력층 , 출력층 , 은닉층\n으로 구성모든 비선형 함수 \n학습이 가능알고리즘을 최적화\n하기 어려운 학습 \n환경 발생\n심층신경망 (DNN)\nDeep Neural \nNetwork입력층과 출력층 사이에 2개 이상의 은닉층\n들로 이뤄진 인공신경망ANN의 문제점 개선학습환경에 따라 \n높은 시간 복잡도  \n문제 발생\n합성곱신경망 (CNN)', metadata={'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 8}),
  0.70871204),
 (Document(page_content='| 10 | CIS이슈리포트 2022-2 호 ▶인공지능 산업의 value chain 은 ‘AI 플랫폼 공급업체 → AI 어플리케이션 개발 → AI 응용솔루션  \n개발 → 이용자 ’로 구성되며 , 동 산업은 ①성장기 산업, ②대체재로부터의 위협이 낮은 산업, ③기\n술집약적 산업 등의 특징을 가짐\n￮알고리즘 , 하드웨어 기술개발과 응용솔루션 서비스 상용화가 활발히 진행 중인 성장기 산업이며 , \n수요 기업의 요구사항에 따라 운영플랫폼을 선택할 수 있는 구매자의 교섭력이 높은 산업임\n￮직접적인 대체 기술이 없어 대체재

In [72]:
db.save_local("faiss_index")

In [74]:
new_db = FAISS.load_local("faiss_index", ko)

In [82]:
query = "인공지능 산업구조는 어떻게 구성되어 있어?"
docs = new_db.similarity_search_with_relevance_scores(query, k=3) # 유사도 기반 
print(f"질문 : {query}")
for i in range(len(docs)):
    print(f"{i+1}번째 유사 문서 유사도 \n{docs[i][1]}")
    print("=="*50)
    print(docs[i][0].page_content)
    print("=="*50)
    print(docs[i][0].metadata)
    print("=="*50)

질문 : 인공지능 산업구조는 어떻게 구성되어 있어?
1번째 유사 문서 유사도 
0.4988649096330402
혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 9 |▶다양한 데이터나 복잡한 자료 속에서 핵심적인 특징을 요약하는 ①데이터 추상화 기술, 방대한 지식
체계를 이용하는 ②빅데이터 기술, 빅데이터를 처리하기 위한 ③고성능 컴퓨팅 기술이 인공지능 
구현의 핵심임
￮데이터를 추상화하는 방법은 크게 인공신경망 (ANN), 심층신경망 (DNN), 합성곱신경망 (CNN) 및 
순환신경망 (RNN) 등으로 구분됨
[인공지능 데이터 추상화 기술]
구분 특징 장점 단점
인공신경망 (ANN)
Artificial Neural 
Network사람의 신경망 원리와 구조를 모방하여 만든 
기계학습 알고리즘으로 , 입력층 , 출력층 , 은닉층
으로 구성모든 비선형 함수 
학습이 가능알고리즘을 최적화
하기 어려운 학습 
환경 발생
심층신경망 (DNN)
Deep Neural 
Network입력층과 출력층 사이에 2개 이상의 은닉층
들로 이뤄진 인공신경망ANN의 문제점 개선학습환경에 따라 
높은 시간 복잡도  
문제 발생
합성곱신경망 (CNN)
{'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 8}
2번째 유사 문서 유사도 
0.48201038491353365
| 10 | CIS이슈리포트 2022-2 호 ▶인공지능 산업의 value chain 은 ‘AI 플랫폼 공급업체 → AI 어플리케이션 개발 → AI 응용솔루션  
개발 → 이용자 ’로 구성되며 , 동 산업은 ①성장기 산업, ②대체재로부터의 위협이 낮은 산업, ③기
술집약적 산업 등의 특징을 가짐
￮알고리즘 , 하드웨어 기술개발과 응용솔루션 서비스 상용화가 활발히 진행 중인 성장기 산업이며 , 
수요 기업의 요구사항에 따라 운영플랫폼을 선택할 수 있는 구매자의 교섭력이 높은 산업임
￮직접적인 대체 기술이 없어 대체재로부터 위협이

In [86]:
query = "인공지능 산업구조는 어떻게 구성되어 있어?"
docs = new_db.max_marginal_relevance_search(query, k=3) # 유사도 뿐만 아니라 다양성도 추구 

print(f"질문 : {query}")
for i in range(len(docs)):
    print(f"{i+1}번째 유사 문서 유사도 \n{docs[i].page_content}")
    print("=="*50)
    print(docs[i].metadata)
    print("=="*50)

질문 : 인공지능 산업구조는 어떻게 구성되어 있어?
1번째 유사 문서 유사도 
혁신성장 정책금융 동향 : ICT 산업을 중심으로
  CIS이슈리포트 2022-2 호 | 9 |▶다양한 데이터나 복잡한 자료 속에서 핵심적인 특징을 요약하는 ①데이터 추상화 기술, 방대한 지식
체계를 이용하는 ②빅데이터 기술, 빅데이터를 처리하기 위한 ③고성능 컴퓨팅 기술이 인공지능 
구현의 핵심임
￮데이터를 추상화하는 방법은 크게 인공신경망 (ANN), 심층신경망 (DNN), 합성곱신경망 (CNN) 및 
순환신경망 (RNN) 등으로 구분됨
[인공지능 데이터 추상화 기술]
구분 특징 장점 단점
인공신경망 (ANN)
Artificial Neural 
Network사람의 신경망 원리와 구조를 모방하여 만든 
기계학습 알고리즘으로 , 입력층 , 출력층 , 은닉층
으로 구성모든 비선형 함수 
학습이 가능알고리즘을 최적화
하기 어려운 학습 
환경 발생
심층신경망 (DNN)
Deep Neural 
Network입력층과 출력층 사이에 2개 이상의 은닉층
들로 이뤄진 인공신경망ANN의 문제점 개선학습환경에 따라 
높은 시간 복잡도  
문제 발생
합성곱신경망 (CNN)
{'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 8}
2번째 유사 문서 유사도 
구분 구조
뉴런Ÿ세포체의 자극이 임계치를 넘으면 신경전달물질 발화
인공지능
(퍼셉트론 )Ÿ활성함수의 계산 결과를 출력
자료: 디지에코
{'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 7}
3번째 유사 문서 유사도 
￮인공지능 기술을 도입하는 산업이 늘어나고 있는 점, 인공지능 분야에 진출하는 스타트업의 증가에 
따라 산업의 기술경쟁력이 높아지는 점 등은 시장성장에  촉진요인으로 작용할 전망임
￮국내 인공지능 기술에 대한 완성도가 높지 않아 국내기술의 도입처가 제한적인 점, 인공지능이 
인간의 고용 영역을 침범할 수 있고 기술의 불안

In [88]:
query = "5G 이동통신 시스템은?"
docs = new_db.max_marginal_relevance_search(query, k=3, fetch_k = 10, lambda_mult=0.3) # 유사도 뿐만 아니라 다양성도 추구 

print(f"질문 : {query}")
for i in range(len(docs)):
    print("=="*50)
    print(f"{i+1}번째 유사 문서 유사도 \n{docs[i].page_content}")
    print("=="*50)
    print(docs[i].metadata)

질문 : 5G 이동통신 시스템은?
1번째 유사 문서 유사도 
확장, ②스몰셀 (Small cell)을 도입한 기지국 , ③다중안테나 송수신 (Massive MIMO), ④네트워크 
슬라이싱 (Network Slicing) 등의 기술을 도입함
[5G 주요 요소기술 특징]
자료: 삼정 KPMG
{'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 5}
2번째 유사 문서 유사도 
| 4 | CIS이슈리포트 2022-2 호 [혁신성장 ICT 산업 정책금융 공급 현황]
                                                            (단위: 억 원, 괄호는 점유율 %)
구분 2017 년 말 2018 년 말 2019 년 말 2020 년 말 2021 년 말
혁신성장 ICT 산업45,075 72,799 81,805 139,687 169,089 
(18.7) (20.7) (18.5) (20.3) (19.8)
정보통신15,658 27,417 39,033 65,324 77,750 
(6.5) (7.8) (8.8) (9.5) (9.1)
전기전자26,637 38,521 35,922 62,856 77,485 
(11.1) (10.9) (8.1) (9.1) (9.1)
센서측정2,780 6,861 6,851 11,506 13,854 
(1.2) (1.9) (1.5) (1.7) (1.6)
혁신성장 정책금융 
총 공급액240,787 351,987 443,180 688,409 854,338 
3. 정보통신 테마 혁신성장 정책금융 현황 및 관련 산업 동향
▶(지원 현황) 정보통신 테마를 구성하는 기술분야별 정책금융 지원 현황 분석결과 , 공급점유율 관점
{'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 3}
3번째 유사 문서 유사도 
￮전체 네트워크 설계역량이 경쟁력의 핵심요소이며 , 지속적인 제품 개발능력과 고객을 만족시키기 
위한 마